## Set up a duckweed genotypes x media growth assay

### Pre-requisites to use this script
1. Precise plate and well positions for your machine defined in Plate_positions.py in the 'utils' subdirectory. 
2. All the parameters defined in the cell 'User paramters'. 
3. Sufficient 24-well plates, sterile media and duckweed plants available. 
4. Jubilee machine set up with Media-dispensing syringe tool (50 mL), Duckweed transfer syringe tool and lab automation bedplate. 


In [8]:
#Importing python libraries downloaded from the internet
import random
import pandas as pd
import os
import json

In [9]:
#Importing python libraries from local files. 
import utils.DuckbotExptSetupUtils as exp
import utils.MachineUtils
from utils.MachineUtils import *
import utils.Plate_positions as pp 

In [10]:
#Establish machine communication 
port = '/dev/ttyACM0' #Check options using  - print([port.name for port in serial.tools.list_ports.comports()])
m = MachineCommunication(port)

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

## 1. Define user parameters

### Labware config 
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [11]:
media_reservoir = { #
'x' : 75, 'y' : 241, #X and Y should get the machine to the center of the reservoir. 
}

#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as?
duckweed_syringe = 3
dispenses_per_syringe_fill = 20
dispense_mL = 1.5 #In mL
vol_conversion = 3.9 #One mL is 3.9 units. 
dispense_offset = dispense_mL * vol_conversion
z_dict = {"zero": 0, #Note that zeros
          "aspirate" : -53, #Measure this for yourself and your labware/toolheads.
          "dispense" : - 34
         }

### Define experimental variables, and desired file names and save locations. 

In [12]:

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498", "sp3484"] # Replace with names for unique duckweed genotypes
media = ["Mock", "Salt"] # Replace with names for unique media
reps  = 3 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

exp_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)

if not os.path.exists(exp_setup_file_path):
    os.mkdir(exp_setup_file_path)     

## 2. Create dataframe with experiment metadata

In [13]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for m in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": m, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}

In [14]:
#Save experimental set up file

os.chdir(exp_setup_file_path)
with open(expt_setup_file_name, 'w') as f:
    json.dump(expt_dict, f)

In [15]:
# path = os.path.join(expt_setup_parent_dir, expt_setup_dir)

# os.chdir(path)
# print(os.getcwd())

In [16]:
#Import from file (in case user wants to make any manual edits to the JSON file after creating it)

with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)
# print (df[0:])


## 3. Label 24-well plates and add to machine

#### Label Plates and add plates to machine

In [17]:
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

1
This experiment requires 1 24-well plate(s)
----
Label a plate with experiment ID or initials and 'plate 1'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


## 2. Dispense media
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [20]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe

num_plates = df.Plate.nunique()
plates = list(range(1, num_plates + 1))
# print(plates)

for p in plates:
    pp.add_well_coords_to_df(p, df)

# print(df)

In [21]:
#Reorganizes dataframe to create machine instructions sorted by media-type

media_df = df.groupby(['media']) #Returns a list of tuples with [0] being the group key and [1] the dataframe
media_dicts = []

for media in media_df:
    well_coords = []
    for index, row in media[1].iterrows():
        this_well = []
        this_well.append(row['x'])
        this_well.append(row['y'])
        well_coords.append(this_well)
    media_dicts.append({'media': media[0], 'well-coords' : well_coords})
# print(media_dicts)


In [ ]:
#Pick up syringe toolhead
m.toolChange(media_syringe)

In [19]:
# Send machine instructions
for media in media_dicts:
    m.moveTo(z=0)
    print(f"Please ensure {media['media']} is available in the machine before continuing.")
    print("Change syringe and/or needle if desired")
     while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
     dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)

Please ensure Mock is available in the machine before continuing.


Enter 'YES' to confirm that the correct media is in position YES


Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E35.10 F6000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 70.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 127.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 51.0
Y = 157.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 70.0
Y = 157.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 89.0
Y = 157.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 108.0
Y = 157.0
G1    E-5.85 F6000.00
Please ensure Salt is available in the machine before continuing.


Enter 'YES' to confirm that the correct media is in position YES


Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E35.10 F6000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 32.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 51.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 89.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 108.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 32.0
Y = 176.0
G1    E-5.85 F6000.00
Prepare to dispense
X = 127.0
Y = 157.0
G1    E-5.85 F6000.00


In [ ]:
m.toolChange(2)
m.setExtruderRelative()
for 

## 5. Transfer duckweed
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [ ]:
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    for index,s in sample_df.iterrows():
        #move to plate and well
        print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))
